In [13]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import glob
from PIL import Image
pd.set_option('display.max_colwidth', None)

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import cv2

In [21]:
def kmeans_segmentation(img, show_image = False):

    vectorized = img.reshape((-1,3))
    vectorized = np.float32(vectorized)
    #print(vectorized.shape)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    K=2
    attempts=10
    ret,label,center=cv2.kmeans(vectorized,K,None,criteria,attempts,cv2.KMEANS_RANDOM_CENTERS)
    label = label.flatten()
    center = np.uint8(center)
    res = center[label.flatten()]
    result_image = res.reshape((img.shape))
    grayscale = cv2.cvtColor(result_image, cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(grayscale, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

    if(show_image):
        plt.imshow(thresh)
        plt.show()
    return thresh

In [22]:
# todo make it read image as binary
def read_true_mask(img_path_str):
    return cv2.cvtColor(cv2.imread(img_path_str),cv2.COLOR_RGB2GRAY)

def get_img(path_str):
    # reads at RGB
    return np.asarray(Image.open(path_str))


def find_metrics(seg_mask_img, org_mask_img):
    f1, iou, pixacc = 0,0,0 
    intersection = np.sum( np.logical_and(seg_mask_img, org_mask_img) )
    union = np.sum ( np.logical_or(seg_mask_img, org_mask_img) )
    cnt_seg = np.count_nonzero(seg_mask_img)
    cnt_org = np.count_nonzero(org_mask_img)
    h,w = seg_mask_img.shape
    cnt_tot = h*w 
    cnt_true = intersection
    cnt_false = cnt_tot - union 

    f1 = 2 * intersection / (cnt_seg + cnt_org) 
    iou = intersection / union
    pixacc = (cnt_true + cnt_false) / cnt_tot

    return f1, iou, pixacc

def print_all(seg_mask_img, org_mask_img):
    f1, iou, pixacc = find_metrics(seg_mask_img, org_mask_img)
    print("f1 = ", f1, " iou = ", iou, " pixacc = ", pixacc)



In [23]:
img_path = './final_dataset/images'
mask_path = './final_dataset/masks'
out_path = './final_dataset/masks_kmeans/'

imgs = glob.glob(os.path.join(img_path, "*.png"))
masks = glob.glob(os.path.join(mask_path, "*.png")) 

imgs = sorted(imgs)
masks = sorted(masks)

num_images = len(masks)

# generate masks and save to file

In [24]:

f1_scores, iou_scores, pixacc_scores = [], [], []


for i in range(0, num_images):
    img = get_img(imgs[i])
    seg_mask_img = kmeans_segmentation(img, show_image=False)
    cv2.imwrite(out_path + 'img_' + str(i).zfill(4) + '.png',  seg_mask_img)

    org_mask_img = read_true_mask(masks[i])
    f1, iou, pixacc = find_metrics(seg_mask_img, org_mask_img)
    f1_scores.append(f1)
    iou_scores.append(iou)
    pixacc_scores.append(pixacc)

    if (i+1)%10 == 0 : 
        print("Processed image", str(i+1))


Processed image 10
Processed image 20
Processed image 30
Processed image 40
Processed image 50
Processed image 60
Processed image 70
Processed image 80
Processed image 90
Processed image 100
Processed image 110
Processed image 120
Processed image 130
Processed image 140
Processed image 150
Processed image 160
Processed image 170
Processed image 180
Processed image 190
Processed image 200


KeyboardInterrupt: 

In [ ]:
sum_f1, sum_iou, sum_pixacc = 0,0,0 

for i in range(0, num_images):
    sum_f1 += f1_scores[i]
    sum_iou += iou_scores[i]
    sum_pixacc += pixacc_scores[i]

print("Average F1 score = ", str(sum_f1 / num_images))
print("Average IOU score = ", str(sum_iou / num_images))
print("Average Pixacc score = ", str(sum_pixacc / num_images))